`LOGISTIC REGRESSION`

In [6]:
## importing neccessary libraries  
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns  
import plotly.express as px   
import plotly.graph_objects as go  
import plotly.io as io  
import patsy 
import statsmodels.api as sm  
from statsmodels.api import Logit 
import pandas as pd     
from scipy import stats
from rich.console import Console 
Console = Console()

In [7]:
## loading the data  
data = pd.read_excel(r'D:\python_data_analysis\Analytics\Practice\projects\healthcare_dataset.xlsx')
## displaying the first five items   
data.head()


,PatientID,SecondSurgeryNeeded,AgeAtFirstSurgery,DiseaseDuration,LengthAffectedIntestine,AlbuminLevel,IBDType,PreviousAbdominalSurgery,SmokingStatus,Comorbidities,TypeOfSurgery,UseOfBiologics,FamilyHistoryOfIBD,Gender,SurgeonExperienceLevel
0,P001,Yes,35,6.8,105,4.3,Crohn's,Yes,Current,No,Open,Yes,No,Female,Specialist
1,P002,Yes,40,4.9,127,3.8,Crohn's,No,Never,No,Open,Yes,Yes,Male,Specialist
2,P003,Yes,31,11.5,135,3.2,Crohn's,Yes,Former,No,Minimally Invasive,Yes,No,Female,Senior
3,P004,Yes,50,20.6,110,3.2,Crohn's,No,Former,Yes,Open,No,Yes,Male,Specialist
4,P005,Yes,35,14.9,151,2.6,Crohn's,Yes,Never,Yes,Open,Yes,No,Male,Mid_Level


In [8]:
## checking the last five  
data.tail()

,PatientID,SecondSurgeryNeeded,AgeAtFirstSurgery,DiseaseDuration,LengthAffectedIntestine,AlbuminLevel,IBDType,PreviousAbdominalSurgery,SmokingStatus,Comorbidities,TypeOfSurgery,UseOfBiologics,FamilyHistoryOfIBD,Gender,SurgeonExperienceLevel
115,P116,No,38,0.1,41,4.6,UC,Yes,Current,No,Minimally Invasive,No,No,Female,Mid_Level
116,P117,No,43,7.2,83,4.3,UC,No,Never,Yes,Minimally Invasive,Yes,No,Male,Senior
117,P118,No,42,3.9,92,2.9,Crohn's,No,Never,Yes,Minimally Invasive,No,No,Male,Senior
118,P119,No,62,17.8,107,3.4,Crohn's,No,Never,Yes,Minimally Invasive,No,No,Male,Mid_Level
119,P120,No,43,16.3,67,4.5,Crohn's,No,Never,Yes,Minimally Invasive,Yes,No,Male,Senior


In [9]:
## checking for shape of the data  
data.shape

(120, 15)

In [10]:
## checking for general info  
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 15 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   PatientID                 120 non-null    object 
 1   SecondSurgeryNeeded       120 non-null    object 
 2   AgeAtFirstSurgery         120 non-null    int64  
 3   DiseaseDuration           120 non-null    float64
 4   LengthAffectedIntestine   120 non-null    int64  
 5   AlbuminLevel              120 non-null    float64
 6   IBDType                   120 non-null    object 
 7   PreviousAbdominalSurgery  120 non-null    object 
 8   SmokingStatus             120 non-null    object 
 9   Comorbidities             120 non-null    object 
 10  TypeOfSurgery             120 non-null    object 
 11  UseOfBiologics            120 non-null    object 
 12  FamilyHistoryOfIBD        120 non-null    object 
 13  Gender                    120 non-null    object 
 14  SurgeonExp

In [11]:
## checking for missing values    
data.isnull().mean()*100

PatientID                   0.0
SecondSurgeryNeeded         0.0
AgeAtFirstSurgery           0.0
DiseaseDuration             0.0
LengthAffectedIntestine     0.0
AlbuminLevel                0.0
IBDType                     0.0
PreviousAbdominalSurgery    0.0
SmokingStatus               0.0
Comorbidities               0.0
TypeOfSurgery               0.0
UseOfBiologics              0.0
FamilyHistoryOfIBD          0.0
Gender                      0.0
SurgeonExperienceLevel      0.0
dtype: float64

In [12]:
## checking for duplicates  
data.duplicated(keep='first').sum()

0

* Exploratory data Analysis

In [13]:
## checking for decscriptive statisics  
data.describe().T

,count,mean,std,min,25%,50%,75%,max
AgeAtFirstSurgery,120.0,39.841667,10.919103,-2.0,33.000,40.00,47.00,66.0
DiseaseDuration,120.0,8.206667,5.164418,0.1,3.875,8.25,11.95,20.6
LengthAffectedIntestine,120.0,100.425000,31.546594,35.0,76.750,99.50,121.50,181.0
AlbuminLevel,120.0,3.747500,0.544331,2.6,3.400,3.70,4.20,5.0


* Choosing the Base Category for each categorical column
  

In [14]:

# IBDType: Base case = Ulcerative Colitis (UC)
data['IBDType'] = pd.Categorical(data['IBDType'], categories=['UC', "Crohn's"])

# SurgeonExperienceLevel: Base case = Senior
data['SurgeonExperienceLevel'] = pd.Categorical(data['SurgeonExperienceLevel'],
                                                categories=['Senior', 'Mid_Level', 'Specialist'])

# PreviousAbdominalSurgery: Base case = No
data['PreviousAbdominalSurgery'] = pd.Categorical(data['PreviousAbdominalSurgery'],
                                                categories=['No', 'Yes'])

# SmokingStatus: Base case = Never
data['SmokingStatus'] = pd.Categorical(data['SmokingStatus'], categories=['Never', 'Former', 'Current'])

# Comorbidities: Base case = No
data['Comorbidities'] = pd.Categorical(data['Comorbidities'], categories=['No', 'Yes'])

# TypeOfSurgery: Base case = Minimally Invasive
data['TypeOfSurgery'] = pd.Categorical(data['TypeOfSurgery'],
                                       categories=['Minimally Invasive', 'Open'])

# UseOfBiologics: Base case = No
data['UseOfBiologics'] = pd.Categorical(data['UseOfBiologics'], categories=['No', 'Yes'])

# FamilyHistoryOfIBD: Base case = No
data['FamilyHistoryOfIBD'] = pd.Categorical(data['FamilyHistoryOfIBD'], categories=['No', 'Yes'])

# Gender: Base case = Female
data['Gender'] = pd.Categorical(data['Gender'], categories=['Female', 'Male'])

# SecondSurgeryNeeded: Base case 
data['SecondSurgeryNeeded'] = pd.Categorical(data['SecondSurgeryNeeded'], categories=['No', 'Yes'])


In [15]:
## checking the dytes  
data.dtypes

PatientID                     object
SecondSurgeryNeeded         category
AgeAtFirstSurgery              int64
DiseaseDuration              float64
LengthAffectedIntestine        int64
AlbuminLevel                 float64
IBDType                     category
PreviousAbdominalSurgery    category
SmokingStatus               category
Comorbidities               category
TypeOfSurgery               category
UseOfBiologics              category
FamilyHistoryOfIBD          category
Gender                      category
SurgeonExperienceLevel      category
dtype: object

In [16]:
## printing the unique counts in each column  
categorical_columns = data.select_dtypes(include=['category']).columns
## creating a dictionary where the data we be stored  
column_dictionary = {}
## looping through the categorical columns  
for column in categorical_columns: 
    column_dictionary[column] = data[column].value_counts()
for key , value in column_dictionary.items():  
    print(f"{key}")
    print(f"{value}")
    print()

SecondSurgeryNeeded
SecondSurgeryNeeded
No     60
Yes    60
Name: count, dtype: int64

IBDType
IBDType
Crohn's    72
UC         48
Name: count, dtype: int64

PreviousAbdominalSurgery
PreviousAbdominalSurgery
No     95
Yes    25
Name: count, dtype: int64

SmokingStatus
SmokingStatus
Never      61
Current    32
Former     27
Name: count, dtype: int64

Comorbidities
Comorbidities
No     63
Yes    57
Name: count, dtype: int64

TypeOfSurgery
TypeOfSurgery
Open                  62
Minimally Invasive    58
Name: count, dtype: int64

UseOfBiologics
UseOfBiologics
No     60
Yes    60
Name: count, dtype: int64

FamilyHistoryOfIBD
FamilyHistoryOfIBD
No     104
Yes     16
Name: count, dtype: int64

Gender
Gender
Female    60
Male      60
Name: count, dtype: int64

SurgeonExperienceLevel
SurgeonExperienceLevel
Specialist    52
Senior        40
Mid_Level     28
Name: count, dtype: int64



In [17]:
## By Age 
import plotly.express as px

fig = px.box(
    data,
    x='Gender',
    y='AgeAtFirstSurgery',
    color='Gender',
    title='Distribution of Age At first Surgery by Gender',
    labels={
        'Gender': 'Gender',
        'AgeAtFirstSurgery': 'Age At first Surgery'
    },
    template='plotly_white'
)

# Update layout for better readability
fig.update_layout(
    title_font=dict(family='Arial', size=22, color='darkblue'),
    font=dict(family='Arial', size=14, color='black'),
    xaxis_title_font=dict(family='Arial', size=16, color='black'),
    yaxis_title_font=dict(family='Arial', size=16, color='black')
)

fig.show()


* Test for Independence

In [18]:
## test for Indepeendence 
data.head()

,PatientID,SecondSurgeryNeeded,AgeAtFirstSurgery,DiseaseDuration,LengthAffectedIntestine,AlbuminLevel,IBDType,PreviousAbdominalSurgery,SmokingStatus,Comorbidities,TypeOfSurgery,UseOfBiologics,FamilyHistoryOfIBD,Gender,SurgeonExperienceLevel
0,P001,Yes,35,6.8,105,4.3,Crohn's,Yes,Current,No,Open,Yes,No,Female,Specialist
1,P002,Yes,40,4.9,127,3.8,Crohn's,No,Never,No,Open,Yes,Yes,Male,Specialist
2,P003,Yes,31,11.5,135,3.2,Crohn's,Yes,Former,No,Minimally Invasive,Yes,No,Female,Senior
3,P004,Yes,50,20.6,110,3.2,Crohn's,No,Former,Yes,Open,No,Yes,Male,Specialist
4,P005,Yes,35,14.9,151,2.6,Crohn's,Yes,Never,Yes,Open,Yes,No,Male,Mid_Level


In [19]:
## checking for independence  
stats.chi2_contingency(pd.crosstab(data['SecondSurgeryNeeded'],data['SmokingStatus']))


Chi2ContingencyResult(statistic=7.813524590163935, pvalue=0.02010549144359778, dof=2, expected_freq=array([[30.5, 13.5, 16. ],
       [30.5, 13.5, 16. ]]))

In [20]:
## test for independence for IBDType and  TypeOfSurgery 
stats.chi2_contingency(pd.crosstab(data['SmokingStatus'],data['Comorbidities']))

Chi2ContingencyResult(statistic=3.971540874043029, pvalue=0.13727481176805686, dof=2, expected_freq=array([[32.025, 28.975],
       [14.175, 12.825],
       [16.8  , 15.2  ]]))

* Finding 
  - The finding indicates that there exist an association between the SecondSurgeryNeeded and SmokingStatus
  - For  SmokingStatus and  Comorbidities there is no associattion at all as the p values are soo high

* Comparing Means

In [21]:
## comparing the means between the different levels of IBDType and LengthAffectedIntestine
stats.ttest_ind(data.loc[data['IBDType']=="Crohn's",'LengthAffectedIntestine'], 
                data.loc[data['IBDType']=="UC",'LengthAffectedIntestine'] 
                )


TtestResult(statistic=3.685771283649253, pvalue=0.0003458202770806173, df=118.0)

In [22]:
stats.ttest_ind(data.loc[data['IBDType']=="Crohn's",'AlbuminLevel'], 
                data.loc[data['IBDType']=="UC",'AlbuminLevel'] 
                )

TtestResult(statistic=-3.921343102156887, pvalue=0.0001482844597986628, df=118.0)

In [23]:
stats.ttest_ind(data.loc[data['PreviousAbdominalSurgery']=="No",'AlbuminLevel'], 
                data.loc[data['PreviousAbdominalSurgery']=="Yes",'AlbuminLevel'] 
                )

TtestResult(statistic=0.6539772445709465, pvalue=0.514398956649805, df=118.0)

In [24]:
## performng a one way anova  
stats.f_oneway(data.loc[data['SmokingStatus']=='Never','LengthAffectedIntestine'], 
               data.loc[data['SmokingStatus']=='Current','LengthAffectedIntestine'],   
               data.loc[data['SmokingStatus']=='Former','LengthAffectedIntestine']   
               )


F_onewayResult(statistic=2.8112154956202495, pvalue=0.0641996246012496)

* Findings 
  - The results show that there is statiscally significant difference for the means of the two different lavels

`ODDS RATIO`
- To understand odds ratio , we have to know the meaning of probability , simplify probabilility is a value given the occurence of an eveent 

`ODDS`
- refers to the  ratio of probabililty of event occuring to compared to it not occuring

`ODDS RATIO`
- it refers to the statiscal measure used to quantify the strength of an association between the exposure aand an outcome

In [25]:
## example  , lets take the a probability of an occurence of an event is 5 in 15 attempts   
p = 5/15
## then the odds would be  
odds = p/(1-p)
odds

0.49999999999999994

In [26]:
## then two impelement odds ratio we need to 
p_ufair = 0.7
p_fair = 0.5   
## calucate the odds foer each  
odds_for_p_unfair = p_ufair/(1-p_ufair)
## claculate the odds for p fair  
odds_for_p_fair = p_fair / (1-p_fair)

In [27]:
## calculate the odds ratio 
odds_ratio = odds_for_p_unfair / odds_for_p_fair
odds_ratio

2.333333333333333

In [28]:
## calculting the odds increase  
odd_increase = odds_ratio - 1
odd_increase


1.333333333333333

* Interpretations  
  - The interpratations will be there is 133.3% increase in the odds of the unfair coin compared to the fair coin

`Modelling the Logistic Regression starting with one continous numerical dependent variable`

In [29]:
##  Generating a dummy variabless for the yes group   
data['SecondSurgeryNeededEncoded'] = pd.get_dummies(data['SecondSurgeryNeeded']).Yes.astype(int)
## checkng the data  
data.head()


,PatientID,SecondSurgeryNeeded,AgeAtFirstSurgery,DiseaseDuration,LengthAffectedIntestine,AlbuminLevel,IBDType,PreviousAbdominalSurgery,SmokingStatus,Comorbidities,TypeOfSurgery,UseOfBiologics,FamilyHistoryOfIBD,Gender,SurgeonExperienceLevel,SecondSurgeryNeededEncoded
0,P001,Yes,35,6.8,105,4.3,Crohn's,Yes,Current,No,Open,Yes,No,Female,Specialist,1
1,P002,Yes,40,4.9,127,3.8,Crohn's,No,Never,No,Open,Yes,Yes,Male,Specialist,1
2,P003,Yes,31,11.5,135,3.2,Crohn's,Yes,Former,No,Minimally Invasive,Yes,No,Female,Senior,1
3,P004,Yes,50,20.6,110,3.2,Crohn's,No,Former,Yes,Open,No,Yes,Male,Specialist,1
4,P005,Yes,35,14.9,151,2.6,Crohn's,Yes,Never,Yes,Open,Yes,No,Male,Mid_Level,1


In [30]:
import plotly.express as px

fig = px.scatter(
    data_frame=data,
    x='LengthAffectedIntestine',
    y='SecondSurgeryNeededEncoded',
    title='Probability of Second Surgery Needed vs. Length of Affected Intestine',
    labels={
        'LengthAffectedIntestine': 'Length of Affected Intestine (cm)',
        'SecondSurgeryNeededEncoded': 'Second Surgery Needed (Encoded)'
    },
    template='plotly_white'
)

# Enhance marker style for better visibility
fig.update_traces(
    marker=dict(
        size=10,
        color='darkred',
        line=dict(width=1, color='black')
    ),
    mode='markers'
)

# Update layout for clearer text and overall aesthetics
fig.update_layout(
    title_font=dict(family='Arial', size=24, color='darkblue'),
    font=dict(family='Arial', size=16, color='black'),
    xaxis_title_font=dict(family='Arial', size=18, color='black'),
    yaxis_title_font=dict(family='Arial', size=18, color='black'),
    plot_bgcolor='white',
    paper_bgcolor='white'
)

fig.show()


In [31]:
## creating dmatrices  
y , X = patsy.dmatrices('SecondSurgeryNeeded~LengthAffectedIntestine',data=data)

In [32]:
## changing the oblect to numpy array   
y = np.array(y[:,1]) 
X = np.array(X)

In [33]:
## passing the data to ols   
log_model_1 = Logit(y,X).fit()

Optimization terminated successfully.
         Current function value: 0.343856
         Iterations 7


In [34]:
## uncovering the results  
log_model_1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:                  120
Model:                          Logit   Df Residuals:                      118
Method:                           MLE   Df Model:                            1
Date:                Wed, 26 Feb 2025   Pseudo R-squ.:                  0.5039
Time:                        17:04:53   Log-Likelihood:                -41.263
converged:                       True   LL-Null:                       -83.178
Covariance Type:            nonrobust   LLR p-value:                 5.392e-20
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -9.7765      1.838     -5.319      0.000     -13.379      -6.174
x1             0.0974      0.018      5.392      0.000       0.062       0.133
==============================================================================
"""

* Findings 
  - The first thing we look at is the R squared = 0.50 approximatley 50.3% of the variation in the log model is explained by the explanatory variable
  - After fitting the model on one continous numerical variable , we look at the  p value is statiscally signficant meaning its a good predictor of the depent 
  - The  coefficent of X1 = 0.0974 which is in a logarithmic scale so we need to exponentiate them
  - 

In [35]:
## getting the parameters and exponentiate them  
odds_for_affected_intestine_length=np.exp(log_model_1.params[1])
## calculate the odds increase  
odds_increase = odds_for_affected_intestine_length - 1
np.round(odds_increase*100,3)

10.226

* Interpretation of the coefficient of the length of Affected Intestine length
  - The odds ratio mean , for each 1 centimetre increase in thee length of affected intestine , the odds of needing a secoond surgery increase by a factor of 1.022 which is 10.22% compared to the base group

In [36]:
## probability of a nedding a second surgery given length of 120 cm  
log_model_1.predict([1,120])
## so the probability of needing a  second surgery given the length of the affected instettine is 0.87 , meaning its so high

array([0.87074719])

In [37]:
## creating an array of affected intestine lengths and visualize them          
affectedIntestineleength = np.linspace(data['LengthAffectedIntestine'].min(),data['LengthAffectedIntestine'].max(),num=200)
## adding a constant to the length of the affected instetine  
affectedIntestineleength_constant_added = sm.add_constant(affectedIntestineleength)
## new predictions  
new_affected_line_predictions = log_model_1.predict(affectedIntestineleength_constant_added)



In [38]:
import plotly.graph_objects as go

fig = go.Figure()

# Adding the model prediction trace (line)
fig.add_trace(go.Scatter(
    x=affectedIntestineleength,
    y=new_affected_line_predictions,
    mode='lines',
    name='Model Prediction',
    line=dict(color='blue', width=3)
))

# Adding the observed data trace (markers)
fig.add_trace(go.Scatter(
    x=data['LengthAffectedIntestine'].tolist(),
    y=data['SecondSurgeryNeededEncoded'].tolist(),
    mode='markers',
    name='Observed Data',
    marker=dict(color='red', size=8, line=dict(width=1, color='black'))
))

# Update layout with enhanced visual settings
fig.update_layout(
    title='Probability of Second Surgery Needed vs. Length of Affected Intestine [cm]',
    title_font=dict(family='Arial', size=24, color='darkblue'),
    xaxis=dict(
        title='Length of Affected Intestine (cm)',
        title_font=dict(family='Arial', size=18, color='black'),
        tickfont=dict(family='Arial', size=14, color='black'),
        gridcolor='lightgray'
    ),
    yaxis=dict(
        title='Second Surgery Needed (Encoded)',
        title_font=dict(family='Arial', size=18, color='black'),
        tickfont=dict(family='Arial', size=14, color='black'),
        gridcolor='lightgray'
    ),
    plot_bgcolor='white',
    paper_bgcolor='white',
    legend=dict(font=dict(family='Arial', size=14, color='black'))
)

fig.show()


* Displaying the underlying Z distribution

In [39]:

# Z score and critical value
z_score = 5.392
z_critical = stats.norm.ppf(0.975)

# x values for the normal distribution plot
xvals = np.linspace(stats.norm.ppf(0.005), stats.norm.ppf(0.995), num=100)

# Create the figure
fig = go.Figure()

# Add trace for the normal distribution (PDF)
fig.add_trace(go.Scatter(
    x=xvals,
    y=stats.norm.pdf(xvals),
    mode='lines',
    name='Normal PDF',
    line=dict(color='blue', width=3)
))

# Add vertical line for the positive z critical value with annotation
fig.add_vline(
    x=z_critical,
    line_dash='dash',
    line_color='red',
    annotation_text=f"z Critical = {z_critical:.2f}",
    annotation_position="top right"
)

# Add vertical line for the negative z critical value with annotation
fig.add_vline(
    x=-z_critical,
    line_dash='dash',
    line_color='red',
    annotation_text=f"-z Critical = {z_critical:.2f}",
    annotation_position="top left"
)

# Add vertical line for the positive observed z score with annotation
fig.add_vline(
    x=z_score,
    line_dash='dash',
    line_color='green',
    annotation_text=f"z Score = {z_score:.2f}",
    annotation_position="bottom right"
)

# Add vertical line for the negative observed z score with annotation
fig.add_vline(
    x=-z_score,
    line_dash='dash',
    line_color='green',
    annotation_text=f"-z Score = {z_score:.2f}",
    annotation_position="bottom left"
)

# Update layout with enhanced fonts, labels, and background
fig.update_layout(
    title='Normal Distribution with Z Critical and Z Score',
    title_font=dict(family='Arial', size=24, color='darkblue'),
    xaxis_title='Z Value',
    xaxis=dict(
        title_font=dict(family='Arial', size=18, color='black'),
        tickfont=dict(family='Arial', size=14, color='black'),
        gridcolor='lightgray'
    ),
    yaxis_title='Probability Density',
    yaxis=dict(
        title_font=dict(family='Arial', size=18, color='black'),
        tickfont=dict(family='Arial', size=14, color='black'),
        gridcolor='lightgray'
    ),
    font=dict(family='Arial', size=16, color='black'),
    plot_bgcolor='white',
    paper_bgcolor='white',
    legend=dict(font=dict(family='Arial', size=14, color='black'))
)

fig.show()


* Logistic Regresssion with a Categorical Indepedent variable

In [40]:
data.head()

,PatientID,SecondSurgeryNeeded,AgeAtFirstSurgery,DiseaseDuration,LengthAffectedIntestine,AlbuminLevel,IBDType,PreviousAbdominalSurgery,SmokingStatus,Comorbidities,TypeOfSurgery,UseOfBiologics,FamilyHistoryOfIBD,Gender,SurgeonExperienceLevel,SecondSurgeryNeededEncoded
0,P001,Yes,35,6.8,105,4.3,Crohn's,Yes,Current,No,Open,Yes,No,Female,Specialist,1
1,P002,Yes,40,4.9,127,3.8,Crohn's,No,Never,No,Open,Yes,Yes,Male,Specialist,1
2,P003,Yes,31,11.5,135,3.2,Crohn's,Yes,Former,No,Minimally Invasive,Yes,No,Female,Senior,1
3,P004,Yes,50,20.6,110,3.2,Crohn's,No,Former,Yes,Open,No,Yes,Male,Specialist,1
4,P005,Yes,35,14.9,151,2.6,Crohn's,Yes,Never,Yes,Open,Yes,No,Male,Mid_Level,1


In [41]:
y , X = patsy.dmatrices('SecondSurgeryNeeded~SurgeonExperienceLevel',data)

In [42]:
## ## changing the oblect to numpy array   
y = np.array(y[:,1]) 
X = np.array(X)

In [43]:
## print the first five items  
y[:5]

array([1., 1., 1., 1., 1.])

In [44]:
## print the first five for X  
X[:5]

array([[1., 0., 1.],
       [1., 0., 1.],
       [1., 0., 0.],
       [1., 0., 1.],
       [1., 1., 0.]])

In [45]:
## feeding it to the model   
log_model_2 = Logit(y,X).fit()

Optimization terminated successfully.
         Current function value: 0.608917
         Iterations 5


In [46]:
## printing the summary results  
log_model_2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:                  120
Model:                          Logit   Df Residuals:                      117
Method:                           MLE   Df Model:                            2
Date:                Wed, 26 Feb 2025   Pseudo R-squ.:                  0.1215
Time:                        17:04:53   Log-Likelihood:                -73.070
converged:                       True   LL-Null:                       -83.178
Covariance Type:            nonrobust   LLR p-value:                 4.077e-05
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.0986      0.365     -3.009      0.003      -1.814      -0.383
x1             0.9555      0.526      1.816      0.069      -0.076       1.987
x2             2.0015      0.476      4.201      0.000       1.068       2.935
==============================================================================
"""

In [47]:
## getting tthe parameters  
log_model_2.params

array([-1.09861229,  0.95551145,  2.00148   ])

In [48]:
## exponaentate 
np.exp(log_model_2.params)

array([0.33333333, 2.6       , 7.4       ])

* Interpretation of the coefficients  
  - lets start with the x1=2.6 which was found be exponenting the coefficeints , so its its above 1 , that meeans there is an association , so it would be interpretated ,the odds of needing a second surgery increases by a factor of 2.6 compared to the based group senior ,meaning a patient who was treated by a mid level surgeon , his or hers odds of needing a second surgery it increase by 16 % compared to when a patient wa who used thee senior surgeon

In [49]:
## try to use the formula aspect  
log_model = sm.Logit.from_formula('SecondSurgeryNeededEncoded~SurgeonExperienceLevel', data).fit()

Optimization terminated successfully.
         Current function value: 0.608917
         Iterations 5


In [50]:
## results  
log_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               Logit Regression Results                               
======================================================================================
Dep. Variable:     SecondSurgeryNeededEncoded   No. Observations:                  120
Model:                                  Logit   Df Residuals:                      117
Method:                                   MLE   Df Model:                            2
Date:                        Wed, 26 Feb 2025   Pseudo R-squ.:                  0.1215
Time:                                17:04:53   Log-Likelihood:                -73.070
converged:                               True   LL-Null:                       -83.178
Covariance Type:                    nonrobust   LLR p-value:                 4.077e-05
========================================================================================================
                                           coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
Intercept                               -1.0986      0.365     -3.009      0.003      -1.814      -0.383
SurgeonExperienceLevel[T.Mid_Level]      0.9555      0.526      1.816      0.069      -0.076       1.987
SurgeonExperienceLevel[T.Specialist]     2.0015      0.476      4.201      0.000       1.068       2.935
========================================================================================================
"""

In [51]:
## getting the confidence intervals 
np.exp(log_model.conf_int())

,0,1
Intercept,0.162954,0.681857
SurgeonExperienceLevel[T.Mid_Level],0.926918,7.292984
SurgeonExperienceLevel[T.Specialist],2.908417,18.828110


In [52]:
## getting the p values  
np.exp(log_model.params)

Intercept                               0.333333
SurgeonExperienceLevel[T.Mid_Level]     2.600000
SurgeonExperienceLevel[T.Specialist]    7.400000
dtype: float64

* Logistic Regression with a continous numeric and a categorical variable

In [53]:
data.head()

,PatientID,SecondSurgeryNeeded,AgeAtFirstSurgery,DiseaseDuration,LengthAffectedIntestine,AlbuminLevel,IBDType,PreviousAbdominalSurgery,SmokingStatus,Comorbidities,TypeOfSurgery,UseOfBiologics,FamilyHistoryOfIBD,Gender,SurgeonExperienceLevel,SecondSurgeryNeededEncoded
0,P001,Yes,35,6.8,105,4.3,Crohn's,Yes,Current,No,Open,Yes,No,Female,Specialist,1
1,P002,Yes,40,4.9,127,3.8,Crohn's,No,Never,No,Open,Yes,Yes,Male,Specialist,1
2,P003,Yes,31,11.5,135,3.2,Crohn's,Yes,Former,No,Minimally Invasive,Yes,No,Female,Senior,1
3,P004,Yes,50,20.6,110,3.2,Crohn's,No,Former,Yes,Open,No,Yes,Male,Specialist,1
4,P005,Yes,35,14.9,151,2.6,Crohn's,Yes,Never,Yes,Open,Yes,No,Male,Mid_Level,1


In [54]:
log_modell_1 = sm.Logit.from_formula('SecondSurgeryNeededEncoded~LengthAffectedIntestine+SurgeonExperienceLevel+AgeAtFirstSurgery+DiseaseDuration+AlbuminLevel+IBDType+TypeOfSurgery', data).fit()

Optimization terminated successfully.
         Current function value: 0.088435
         Iterations 11


In [55]:
## printing the summary results  
log_modell_1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               Logit Regression Results                               
======================================================================================
Dep. Variable:     SecondSurgeryNeededEncoded   No. Observations:                  120
Model:                                  Logit   Df Residuals:                      111
Method:                                   MLE   Df Model:                            8
Date:                        Wed, 26 Feb 2025   Pseudo R-squ.:                  0.8724
Time:                                17:04:53   Log-Likelihood:                -10.612
converged:                               True   LL-Null:                       -83.178
Covariance Type:                    nonrobust   LLR p-value:                 2.029e-27
========================================================================================================
                                           coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
Intercept                               -0.9132      7.203     -0.127      0.899     -15.031      13.205
SurgeonExperienceLevel[T.Mid_Level]      1.2830      1.563      0.821      0.412      -1.780       4.346
SurgeonExperienceLevel[T.Specialist]     1.5021      1.611      0.932      0.351      -1.656       4.660
IBDType[T.Crohn's]                       5.9525      2.785      2.137      0.033       0.493      11.412
TypeOfSurgery[T.Open]                    4.6247      2.044      2.262      0.024       0.618       8.632
LengthAffectedIntestine                  0.1152      0.042      2.758      0.006       0.033       0.197
AgeAtFirstSurgery                       -0.3676      0.147     -2.499      0.012      -0.656      -0.079
DiseaseDuration                          0.2824      0.207      1.366      0.172      -0.123       0.687
AlbuminLevel                            -1.2381      1.467     -0.844      0.399      -4.114       1.638
========================================================================================================

Possibly complete quasi-separation: A fraction 0.41 of observations can be
perfectly predicted. This might indicate that there is complete
quasi-separation. In this case some parameters will not be identified.
"""

In [56]:
## getting the exponentiate  
np.exp(log_modell_1.params)

Intercept                                 0.401247
SurgeonExperienceLevel[T.Mid_Level]       3.607324
SurgeonExperienceLevel[T.Specialist]      4.491305
IBDType[T.Crohn's]                      384.703936
TypeOfSurgery[T.Open]                   101.973093
LengthAffectedIntestine                   1.122113
AgeAtFirstSurgery                         0.692405
DiseaseDuration                           1.326261
AlbuminLevel                              0.289939
dtype: float64

* Variance Infaltion factor  
  

In [57]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [58]:
y , X = patsy.dmatrices('SecondSurgeryNeeded~SurgeonExperienceLevel+LengthAffectedIntestine+AgeAtFirstSurgery',data)

In [59]:
## converting to numpy array  
X=np.array(X)

In [60]:
# Convert numpy array X to a pandas DataFrame
X_df = pd.DataFrame(X, columns=['Intercept', 'SurgeonExperienceLevel_Specialist', 'SurgeonExperienceLevel_Senior', 'LengthAffectedIntestine', 'AgeAtFirstSurgery'])

# Calculate VIF
vif = pd.Series([variance_inflation_factor(X_df.values, i) for i in range(X_df.shape[1])], index=X_df.columns)
vif

Intercept                            33.420503
SurgeonExperienceLevel_Specialist     1.333157
SurgeonExperienceLevel_Senior         1.576224
LengthAffectedIntestine               1.065823
AgeAtFirstSurgery                     1.167887
dtype: float64

In [61]:
new_data = pd.DataFrame({
	'LengthAffectedIntestine': [60],
	'SurgeonExperienceLevel': ['Mid_Level'],
	'AgeAtFirstSurgery': [35],
	'DiseaseDuration': [5],  
	'AlbuminLevel': [3.5], 
	'IBDType': ["Crohn's"],
	'TypeOfSurgery': ['Open'] 
})

# Ensure categorical variables are properly encoded
new_data['SurgeonExperienceLevel'] = pd.Categorical(new_data['SurgeonExperienceLevel'], categories=['Senior', 'Mid_Level', 'Specialist'])
new_data['IBDType'] = pd.Categorical(new_data['IBDType'], categories=['UC', "Crohn's"])
new_data['TypeOfSurgery'] = pd.Categorical(new_data['TypeOfSurgery'], categories=['Minimally Invasive', 'Open'])

predictions = log_modell_1.predict(new_data)
predictions

0    0.88824
dtype: float64

In [62]:
from statsmodels.stats.anova import anova_lm

In [63]:
log_modell_1.params

Intercept                              -0.913179
SurgeonExperienceLevel[T.Mid_Level]     1.282966
SurgeonExperienceLevel[T.Specialist]    1.502143
IBDType[T.Crohn's]                      5.952474
TypeOfSurgery[T.Open]                   4.624709
LengthAffectedIntestine                 0.115214
AgeAtFirstSurgery                      -0.367584
DiseaseDuration                         0.282364
AlbuminLevel                           -1.238084
dtype: float64

In [64]:
log_modell_1.pvalues

Intercept                               0.899120
SurgeonExperienceLevel[T.Mid_Level]     0.411616
SurgeonExperienceLevel[T.Specialist]    0.351178
IBDType[T.Crohn's]                      0.032592
TypeOfSurgery[T.Open]                   0.023686
LengthAffectedIntestine                 0.005812
AgeAtFirstSurgery                       0.012472
DiseaseDuration                         0.171919
AlbuminLevel                            0.398839
dtype: float64